In [1]:
import numpy as np
import os
import cv2
from tensorflow.keras import applications, models, layers

In [7]:
# 데이터 전처리
def get_data(path):
    x = []
    y = []
    dir_list = os.listdir(path)
    for i in range(len(dir_list)-1):
        dir_path = path + "/" + dir_list[i+1]
        dir_name = os.listdir(dir_path)

        for j in range(len(dir_name)):
            full_dir_path = dir_path + "/" + dir_name[j]

            img = cv2.imread(full_dir_path)
            img = cv2.resize(img, (32,32))
            x.append(img)
            y.append(i)
    # print(x)
    # print(y)
    return np.array(x),np.array(y)
#get_data()

In [8]:
# 모델 생성
def make_model() :
    resnet50 = applications.resnet50.ResNet50(include_top=False, weights="imagenet", input_shape=(32,32,3))
    resnet50.trainable = False
    model = models.Sequential()
    model.add(resnet50)
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Dense(512, activation="relu"))

    # 분류 데이터가 몇종류인지
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(loss = "sparse_categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])
    print(model.summary())
    return model

In [9]:
# 한개 테스트용
def get_test():
    test_path = "./test/non/test17.png"
    test_img = cv2.imread(test_path)
    #print(test_img)
    test_img = cv2.resize(test_img, (32,32))
    #test_img = test_img.reshape(32,32,3)
    test_list = []
    test_list.append(test_img)
    test_list = np.array(test_list)
    #print(test_img.shape)
    cv2.imwrite("test123.png",test_img)
    return test_list

In [11]:
# 모델 학습
def learning() :
    train_path = "./data_image"
    test_path = "./test"
    (x_train, y_train)= get_data(train_path)
    (x_test, y_test) = get_data(test_path)
    model = make_model()
    # print("x_train : ",x_train[0])
    # print("x_train.shape : ",x_train[0].shape)
    log = model.fit(x_train, y_train, epochs=2, batch_size=16)
    #model.save("face.h5")
    
    model.evaluate(x_test,y_test)
learning()

TypeError: get_data() takes 0 positional arguments but 1 was given

In [24]:
import cv2
import numpy as np
path = "./tracking/test1.png"

img = cv2.imread(path, cv2.IMREAD_COLOR)

# 색 변경
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

img_hand = cv2.inRange(img_gray,(10), (165))
mask = np.zeros(img.shape).astype(img.dtype)

#경계선 찾음
contours, hierarchy = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# 가장 큰 영역 찾기
max = 0
maxcnt = None
for cnt in contours :
    area = cv2.contourArea(cnt)
    if(max < area) :
        max = area
        maxcnt = cnt


# 경계선 내부 255로 채우기
color = [255, 255, 255]
cv2.fillPoly(mask, [maxcnt], color)
img_hand = cv2.bitwise_and(img, mask)
hull = cv2.convexHull(maxcnt)
#print(hull)

# 도출된 값으로 사각형 그리기
x,y,w,h = cv2.boundingRect(hull)
cv2.rectangle(mask, (x,y), (x+w,y+h), (0,255,0),3 )


cv2.imshow('image', mask)
#cv2.imshow('image', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


-1